Data fetching.

In [3]:
import dicom
from functools import reduce
import json
from matplotlib import pyplot as plt
from operator import add
from os import path
import pandas as pd
import re
import requests

%matplotlib inline

In [4]:
data_list = []

In [6]:
i = 0

while True:
    begin = i * 30 + 1
    end = (i + 1) * 30
    resp = requests.get("https://openi.nlm.nih.gov/retrieve.php?it=x&coll=iu&fields=msh&m={}&n={}".format(begin, end))
    tmp_list = json.loads(resp.text)["list"]
    if len(tmp_list) < 1:
        break
    data_list.extend(tmp_list)
    
    i += 1
    
    if i % 100 == 0:
        print(begin, end)

2971 3000
5971 6000


In [7]:
pattern = re.compile(r"(.*?)([0-9]+)_IM-([^\.]*)\.(.*)")

for data in data_list:
    data["localFilename"] = re.sub(pattern, r"\2/\2_IM-\3.dcm", data["imgLarge"])

In [8]:
dicom_filepaths = !ls /home/a.kondyukov/data/Indianapolis_dicom/**/*
filenames_set = set(dicom_filepaths)

for data in data_list:
    var = "/home/a.kondyukov/data/Indianapolis_dicom/" + data["localFilename"]
    assert var in filenames_set
        
        
new_filenames_set = set(["/home/a.kondyukov/data/Indianapolis_dicom/" + data["localFilename"] for data in data_list])
for path in dicom_filepaths:
    assert path in new_filenames_set, path

In [9]:
majors = [data["MeSH"]["major"] for data in data_list]

In [10]:
majors_flat = reduce(add, majors)

In [11]:
len(majors_flat)

15352

In [12]:
pd.Series(majors_flat).value_counts()

normal                                             2696
Lung/hypoinflation                                  467
Lung/hyperdistention                                318
Cardiomegaly                                        269
Cardiomegaly/mild                                   245
Aorta/tortuous                                      225
Thoracic Vertebrae/degenerative                     212
Spine/degenerative                                  208
Granulomatous Disease                               185
Technical Quality of Image Unsatisfactory           183
Atherosclerosis/aorta                               172
No Indexing                                         172
Thoracic Vertebrae/degenerative/mild                172
Markings/bronchovascular                            132
Calcinosis/aorta                                    121
Aorta, Thoracic/tortuous                            102
Emphysema                                            99
Spondylosis/thoracic vertebrae                  